In [ ]:
#Preliminaries
import os
import numpy as np 
import pandas as pd 
import re
import random
import string
from tqdm import tqdm
from tqdm.autonotebook import tqdm
from sklearn import model_selection
from sklearn import metrics

#Tokenizers from hugging face
import tokenizers

#Pytorch
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.optim import lr_scheduler

#Transformers
import transformers
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup

#Ignore Warnings
import warnings
warnings.filterwarnings("ignore")

# Printing Paths
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Seed Up

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

In [ ]:
seed=2020
seed_everything(seed)

# Config

In [ ]:
MAX_LEN = 124
TRAIN_BATCH_SIZE = 50
VALID_BATCH_SIZE = 32
EPOCHS = 5
ROBERTA_PATH = "../input/roberta-base"
TOKENIZER = tokenizers.ByteLevelBPETokenizer(
    vocab_file=f"{ROBERTA_PATH}/vocab.json", 
    merges_file=f"{ROBERTA_PATH}/merges.txt", 
    lowercase=True,
    add_prefix_space=True
)

# Utils

In [ ]:
class AverageMeter(object):
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [ ]:
class EarlyStopping:
    def __init__(self, patience=7, mode="max", delta=0.001):
        self.patience = patience
        self.counter = 0
        self.mode = mode
        self.best_score = None
        self.early_stop = False
        self.delta = delta
        if self.mode == "min":
            self.val_score = np.Inf
        else:
            self.val_score = -np.Inf

    def __call__(self, epoch_score, model, model_path):

        if self.mode == "min":
            score = -1.0 * epoch_score
        else:
            score = np.copy(epoch_score)

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(epoch_score, model, model_path)
        elif score < self.best_score + self.delta:
            self.counter += 1
            print('EarlyStopping counter: {} out of {}'.format(self.counter, self.patience))
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(epoch_score, model, model_path)
            self.counter = 0

    def save_checkpoint(self, epoch_score, model, model_path):
        if epoch_score not in [-np.inf, np.inf, -np.nan, np.nan]:
            print('Validation score improved ({} --> {}). Saving model!'.format(self.val_score, epoch_score))
            torch.save(model.state_dict(), model_path)
        self.val_score = epoch_score

In [ ]:
def onehot(size, target):
    vec = torch.zeros(size, dtype=torch.long)
    vec[target] = 1.
    return vec

In [ ]:
def jaccard(str1, str2): 
    a = set(str1.lower().split()) 
    b = set(str2.lower().split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

# Model

In [ ]:
class TweetModel(transformers.BertPreTrainedModel):
    def __init__(self, conf):
        super(TweetModel, self).__init__(conf)
        self.roberta = transformers.RobertaModel.from_pretrained(ROBERTA_PATH, config=conf)
        
        self.d0 = nn.Dropout(0.1)
        self.d1 = nn.Dropout(0.1)
        
        self.l0 = nn.Linear(768 * 2, 2)
        self.l1 = nn.Linear(768 * 2, 3)
        
        torch.nn.init.normal_(self.l0.weight, std=0.02)
        torch.nn.init.normal_(self.l1.weight, std=0.02)
    
    def forward(self, ids, mask, token_type_ids):
        _, _, out = self.roberta(
            ids,
            attention_mask=mask,
            token_type_ids=token_type_ids
        )

        out = torch.cat((out[-1], out[-2]), dim=-1)
        
        # Head 1
        x = self.d0(out)
        logits = self.l0(x)
        start_logits, end_logits = logits.split(1, dim=-1)
        start_logits = start_logits.squeeze(-1)
        end_logits = end_logits.squeeze(-1)
        
        # Head 2
        y = self.d1(out)
        y = self.l1(y)
        y = y[:,0,:]
        sentiment = y.squeeze(-2)

        return start_logits, end_logits, sentiment

# Mr.Loss

In [ ]:
def loss_fn(start_logits, end_logits, sent_logits, start_positions, end_positions, sentiments):
    loss_fct = nn.CrossEntropyLoss()
    
    start_loss = loss_fct(start_logits, start_positions)
    end_loss = loss_fct(end_logits, end_positions)
    total_loss = (start_loss + end_loss)
    
    sent_loss = loss_fct(sent_logits, sentiments)
    
    return torch.mean(torch.stack([total_loss, sent_loss]))

# Cooking Up Data

In [ ]:
def process_data(tweet, selected_text, sentiment, tokenizer, max_len):
    tweet = " " + " ".join(str(tweet).split())
    selected_text = " " + " ".join(str(selected_text).split())

    len_st = len(selected_text) - 1
    idx0 = None
    idx1 = None

    for ind in (i for i, e in enumerate(tweet) if e == selected_text[1]):
        if " " + tweet[ind: ind+len_st] == selected_text:
            idx0 = ind
            idx1 = ind + len_st - 1
            break

    char_targets = [0] * len(tweet)
    if idx0 != None and idx1 != None:
        for ct in range(idx0, idx1 + 1):
            char_targets[ct] = 1
    
    tok_tweet = tokenizer.encode(tweet)
    input_ids_orig = tok_tweet.ids
    tweet_offsets = tok_tweet.offsets
    
    target_idx = []
    for j, (offset1, offset2) in enumerate(tweet_offsets):
        if sum(char_targets[offset1: offset2]) > 0:
            target_idx.append(j)
    
    targets_start = target_idx[0]
    targets_end = target_idx[-1]
    
    sentiment = str(sentiment)

    sentiment_id = {
        '2': 1313,
        '0': 2430,
        '1': 7974
    }
    
    input_ids = [0] + [sentiment_id[sentiment]] + [2] + [2] + input_ids_orig + [2]
    token_type_ids = [0, 0, 0, 0] + [0] * (len(input_ids_orig) + 1)
    mask = [1] * len(token_type_ids)
    tweet_offsets = [(0, 0)] * 4 + tweet_offsets + [(0, 0)]
    targets_start += 4
    targets_end += 4

    padding_length = max_len - len(input_ids)
    if padding_length > 0:
        input_ids = input_ids + ([1] * padding_length)
        mask = mask + ([0] * padding_length)
        token_type_ids = token_type_ids + ([0] * padding_length)
        tweet_offsets = tweet_offsets + ([(0, 0)] * padding_length)
            
    return {
        'ids': input_ids,
        'mask': mask,
        'token_type_ids': token_type_ids,
        'targets_start': targets_start,
        'targets_end': targets_end,
        'orig_tweet': tweet,
        'orig_selected': selected_text,
        'sentiment': int(sentiment),
        'offsets': tweet_offsets
    }

In [ ]:
class TweetDataset:
    def __init__(self, tweet, sentiment, selected_text):
        self.tweet = tweet
        self.sentiment = sentiment
        self.selected_text = selected_text
        self.tokenizer = TOKENIZER
        self.max_len = MAX_LEN
    
    def __len__(self):
        return len(self.tweet)

    def __getitem__(self, item):
        data = process_data(
            self.tweet[item], 
            self.selected_text[item],
            self.sentiment[item],
            self.tokenizer,
            self.max_len
        )
        
        #sentiment = onehot(3, self.sentiment[item])

        return {
            'ids': torch.tensor(data["ids"], dtype=torch.long),
            'mask': torch.tensor(data["mask"], dtype=torch.long),
            'token_type_ids': torch.tensor(data["token_type_ids"], dtype=torch.long),
            'targets_start': torch.tensor(data["targets_start"], dtype=torch.long),
            'targets_end': torch.tensor(data["targets_end"], dtype=torch.long),
            'orig_tweet': data["orig_tweet"],
            'orig_selected': data["orig_selected"],
            'sentiment': torch.tensor(data['sentiment'], dtype=torch.long),
            'offsets': torch.tensor(data["offsets"], dtype=torch.long)
        }

# Decoder

In [ ]:
def calculate_jaccard_score(
    original_tweet, 
    target_string, 
    sentiment_val, 
    idx_start, 
    idx_end, 
    offsets,
    verbose=False):
    
    if idx_end < idx_start:
        idx_end = idx_start
    
    filtered_output  = ""
    for ix in range(idx_start, idx_end + 1):
        filtered_output += original_tweet[offsets[ix][0]: offsets[ix][1]]
        if (ix+1) < len(offsets) and offsets[ix][1] < offsets[ix+1][0]:
            filtered_output += " "

    if sentiment_val == "neutral" or len(original_tweet.split()) < 2:
        filtered_output = original_tweet

    #jac = 0
    jac = jaccard(target_string.strip(), filtered_output.strip())
    return jac, filtered_output

# Evaluation

In [ ]:
def eval_fn(data_loader, model, device):
    
    model.eval()
    losses = AverageMeter()
    jaccards = AverageMeter()
    accuracy = AverageMeter()
    
    tk0 = tqdm(data_loader, total=len(data_loader), desc="Validating")
    
    with torch.no_grad():
        for bi, d in enumerate(tk0):
            ids = d["ids"]
            token_type_ids = d["token_type_ids"]
            mask = d["mask"]
            sentiment = d["sentiment"]
            orig_selected = d["orig_selected"]
            orig_tweet = d["orig_tweet"]
            targets_start = d["targets_start"]
            targets_end = d["targets_end"]
            offsets = d["offsets"].cpu().numpy()

            ids = ids.to(device, dtype=torch.long)
            token_type_ids = token_type_ids.to(device, dtype=torch.long)
            mask = mask.to(device, dtype=torch.long)
            
            targets_start = targets_start.to(device, dtype=torch.long)
            targets_end = targets_end.to(device, dtype=torch.long)
            sentiment = sentiment.to(device, dtype=torch.long)

            outputs_start, outputs_end, outputs_sent = model(
                ids=ids,
                mask=mask,
                token_type_ids=token_type_ids
            )
            
            loss = loss_fn(outputs_start, outputs_end, outputs_sent, targets_start, targets_end, sentiment)
            outputs_start = torch.softmax(outputs_start, dim=1).cpu().detach().numpy()
            outputs_end = torch.softmax(outputs_end, dim=1).cpu().detach().numpy()
            outputs_sent = torch.softmax(outputs_sent, dim=1).cpu().detach().numpy().argmax(axis=1)
            
            jaccard_scores = []
            
            for px, tweet in enumerate(orig_tweet):
                selected_tweet = orig_selected[px]
                tweet_sentiment = sentiment[px]
                jaccard_score, _ = calculate_jaccard_score(
                    original_tweet=tweet,
                    target_string=selected_tweet,
                    sentiment_val=tweet_sentiment,
                    idx_start=np.argmax(outputs_start[px, :]),
                    idx_end=np.argmax(outputs_end[px, :]),
                    offsets=offsets[px]
                )
                jaccard_scores.append(jaccard_score)
            
            acc = metrics.accuracy_score(sentiment.cpu().detach().numpy(), outputs_sent)

            jaccards.update(np.mean(jaccard_scores), ids.size(0))
            losses.update(loss.item(), ids.size(0))
            accuracy.update(acc, ids.size(0))
            tk0.set_postfix(loss=losses.avg, jaccard=jaccards.avg,accuracy=accuracy.avg)

            
    return losses.avg, jaccards.avg, accuracy.avg

# Training function

In [ ]:
def train_fn(data_loader, model, optimizer, device, scheduler=None):
    model.train()
    
    losses = AverageMeter()
    jaccards = AverageMeter()
    accuracy = AverageMeter()
    
    tk0 = tqdm(data_loader, total=len(data_loader), desc="Training")
    
    for bi, d in enumerate(tk0):
        ids = d["ids"]
        token_type_ids = d["token_type_ids"]
        mask = d["mask"]
        targets_start = d["targets_start"]
        targets_end = d["targets_end"]
        sentiment = d["sentiment"]
        orig_selected = d["orig_selected"]
        orig_tweet = d["orig_tweet"]
        targets_start = d["targets_start"]
        targets_end = d["targets_end"]
        offsets = d["offsets"]

        ids = ids.to(device, dtype=torch.long)
        token_type_ids = token_type_ids.to(device, dtype=torch.long)
        mask = mask.to(device, dtype=torch.long)
        
        targets_start = targets_start.to(device, dtype=torch.long)
        targets_end = targets_end.to(device, dtype=torch.long)
        sentiment = sentiment.to(device, dtype=torch.long)

        model.zero_grad()
        
        outputs_start, outputs_end, outputs_sent = model(
            ids=ids,
            mask=mask,
            token_type_ids=token_type_ids,
        )
        
        loss = loss_fn(outputs_start, outputs_end, outputs_sent, \
                       targets_start, targets_end, sentiment)
        loss.backward()
        optimizer.step()
        scheduler.step()
        
        outputs_start = torch.softmax(outputs_start, dim=1).cpu().detach().numpy()
        outputs_end = torch.softmax(outputs_end, dim=1).cpu().detach().numpy()
        outputs_sent = torch.softmax(outputs_sent, dim=1).cpu().detach().numpy().argmax(axis=1)
        
        jaccard_scores = []
        for px, tweet in enumerate(orig_tweet):
            selected_tweet = orig_selected[px]
            tweet_sentiment = sentiment[px]
            jaccard_score, _ = calculate_jaccard_score(
                original_tweet=tweet,
                target_string=selected_tweet,
                sentiment_val=tweet_sentiment,
                idx_start=np.argmax(outputs_start[px, :]),
                idx_end=np.argmax(outputs_end[px, :]),
                offsets=offsets[px]
            )
            jaccard_scores.append(jaccard_score)
        
            
        acc = metrics.accuracy_score(sentiment.cpu().detach().numpy(), outputs_sent)

        jaccards.update(np.mean(jaccard_scores), ids.size(0))
        losses.update(loss.item(), ids.size(0))
        accuracy.update(acc, ids.size(0))
        tk0.set_postfix(loss=losses.avg, jaccard=jaccards.avg,accuracy = accuracy.avg)

# Training

In [ ]:
from sklearn import model_selection

dfx = pd.read_csv("/kaggle/input/tweet-sentiment-extraction/train.csv")
dfx = dfx.dropna().reset_index(drop=True)
dfx["kfold"] = -1

dfx = dfx.sample(frac=1,random_state=seed).reset_index(drop=True)

kf = model_selection.StratifiedKFold(n_splits=10,random_state=seed)

for fold, (trn_, val_) in enumerate(kf.split(X=dfx, y=dfx.sentiment.values)):
    print(len(trn_), len(val_))
    dfx.loc[val_, 'kfold'] = fold

In [ ]:
sent = {'negative': 0, 'neutral': 1, 'positive': 2}
dfx['sentiment'] = dfx['sentiment'].map(sent)

In [ ]:
def run(fold):
    df_train = dfx[dfx.kfold != fold].reset_index(drop=True)#.sample(frac= 1, random_state=2020)
    df_valid = dfx[dfx.kfold == fold].reset_index(drop=True)#.sample(frac= 1, random_state=2020)
    
    train_dataset = TweetDataset(
        tweet=df_train.text.values,
        sentiment=df_train.sentiment.values,
        selected_text=df_train.selected_text.values
    )

    train_data_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=TRAIN_BATCH_SIZE,
        num_workers=4
    )

    valid_dataset = TweetDataset(
        tweet=df_valid.text.values,
        sentiment=df_valid.sentiment.values,
        selected_text=df_valid.selected_text.values
    )

    valid_data_loader = torch.utils.data.DataLoader(
        valid_dataset,
        batch_size=VALID_BATCH_SIZE,
        num_workers=2
    )

    device = torch.device("cuda")
    model_config = transformers.RobertaConfig.from_pretrained(ROBERTA_PATH)
    model_config.output_hidden_states = True
    model = TweetModel(conf=model_config)
    model.to(device)

    num_train_steps = int(len(df_train) / TRAIN_BATCH_SIZE * EPOCHS)
    param_optimizer = list(model.named_parameters())
    no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
    optimizer_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.001},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0},
    ]
    optimizer = AdamW(optimizer_parameters, lr=3e-5)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, 
        num_warmup_steps=0, 
        num_training_steps=num_train_steps
    )

    es = EarlyStopping(patience=4, mode="max")
    print(f"Training is Starting for fold={fold}")
    
    # I'm training only for 3 epochs even though I specified 5!!!
    for epoch in range(5):
        train_fn(train_data_loader, model, optimizer, device, scheduler=scheduler)
        loss,jaccard,acc = eval_fn(valid_data_loader, model, device)
        print(f"Jaccard Score = {jaccard}")
        es(jaccard, model, model_path=f"model_{fold}.bin")
        if es.early_stop:
            print("Early stopping")
            break

In [ ]:
run(fold=0)

In [ ]:
run(fold=1)

In [ ]:
run(fold=2)

In [ ]:
run(fold=3)

In [ ]:
run(fold=4)

In [ ]:
run(fold=5)

In [ ]:
run(fold=6)

In [ ]:
run(fold=7)

In [ ]:
run(fold=8)

In [ ]:
run(fold=9)